# A Phosphorus Cycling Model

## Tracer equations

We consider a simple model for the cycling of phosphorus with 3 state variables consisting of phosphate (PO₄) AKA dissolved inorganic phosphorus (DIP), dissolved organic phosphorus (DOP), and particulate organic phosphorus (POP).

The dissolved phases are transported by advection and diffusion whereas the particulate phase sinks rapidly down the water column without any appreciable transport by the circulation.

The governing equations are:

$$\frac{\partial}{\partial t} DIP + \nabla \cdot \left[\boldsymbol{u} + \mathbf{K}\cdot\nabla \right] DIP = -\gamma(DIP) + \kappa_\mathsf{D} \, DOP,$$

$$\frac{\partial}{\partial t} DOP + \nabla \cdot \left[\boldsymbol{u} + \mathbf{K}\cdot \nabla \right] DOP = \sigma \, \gamma(DIP) + \kappa_\mathsf{P} \, POP - \kappa_\mathsf{D} \, DOP,$$

and

$$\frac{\partial}{\partial t} POP + \frac{\partial}{\partial z} \left[w_\mathsf{P} \, POP\right] = (1-\sigma) \, \gamma(DIP) - \kappa_\mathsf{P} \, POP,$$

where $\boldsymbol{u}$ is the fluid velocity and $\mathbf{K}$ is the eddy-diffusion tensor.
Thus, $\nabla \cdot \left[ \boldsymbol{u} - \mathbf{K} \cdot \nabla \right]$ is a differential operator that represents the transport by the ocean circulation.
The function $\gamma(DIP)$ represents the biological uptake of DIP by phytoplankton.

Oxygen participates to this cycle too and satisfies its own tracer equation

$$\frac{\partial}{\partial t} DO_2 + \nabla \cdot \left[\boldsymbol{u} + \mathbf{K}\cdot\nabla \right] DO_2 = -r_{\mathsf{O}_2:\mathsf{P}} \, \kappa_\mathsf{D} \, DOP + \boldsymbol{\Lambda}(DO_2 - [O_2]_{\mathsf{sat}})$$

where $\Lambda$ is the air-sea gas exchange operator.

These tracer equations depend on a number of scalars, that we list below

| Symbol                        | Definition                                                                  |
|:------------------------------|:----------------------------------------------------------------------------|
| $w_\mathsf{P}$                | depth dependent particle sinking speed                                      |
| $\sigma$                      | fraction of the organic matter production allocated to the dissolved phase  |
| $\kappa_\mathsf{D}$           | respiration rate for dissolved organic matter (DOP → DIP)                   |
| $\kappa_\mathsf{P}$           | dissolution rate for particulate organic matter (POP → DOP)                 |
| $r_{\mathsf{O}_2:\mathsf{P}}$ | number of moles of O₂ needed to respire 1 mole of DOP                       |

In [1]:
using AIBECS

Load the circulation and grid

In [2]:
const wet3d, grd, T_Circulation = OCIM1.load()

Loading OCIM1 with JLD2┌ Warning: Over-writing registration of the datadep
│   name = MatricesForJAMES
└ @ DataDeps /home/travis/.julia/packages/DataDeps/LiEdA/src/registration.jl:8
 ✅


([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 1.0 1.0 … 1.0 1.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 1.0 1.0 … 1.0 1.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 1.0 1.0 … 1.0 1.0; 0.0 0.0 … 0.0 0.0]

...

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Dict{String,Any}("XT" => [1.0 3.0 … 357.0 359.0; 1.0 3.0 … 357.0 359.0; … ; 1.0 3.0 … 357.0 359.0; 1.0 3.0 … 357.0 359.0],"Areat" => [8.442828573400822e8 8.442828573400822e8 … 8.442828573400822e8 8.442828573400822e8; 2.53184242418399e9 2.53184242418399e9 … 2.53184242418399e9 2.53184242418399e9; … ; 2.531842424184019e9 2.531842424184019e9 … 2.531842424184019e9 2.531842424184019e9; 8.442828573400851e8 8.442828573400851e8 … 8.442828573400851e8 8.442828573400851e8],"YC" => [-88.02197802197801 -88.0219

Define useful constants and arrays

In [3]:
const iwet = indices_of_wet_boxes(wet3d)
const nb = number_of_wet_boxes(wet3d)
const v = vector_of_volumes(wet3d, grd)
const z = vector_of_depths(wet3d, grd)
const ztop = vector_of_top_depths(wet3d, grd)

200160-element Array{Float64,1}:
    0.0           
    0.0           
    0.0           
    0.0           
    0.0           
    0.0           
    0.0           
    0.0           
    0.0           
    0.0           
    ⋮             
 5116.506284367635
 5116.506284367635
 5116.506284367635
 5116.506284367635
 5116.506284367635
 5116.506284367635
 5116.506284367635
 5116.506284367635
 5116.506284367635

And matrices

In [4]:
const DIV = buildDIV(wet3d, iwet, grd)
const Iabove = buildIabove(wet3d, iwet) ;

### Transport matrices

In [5]:
T_DIP(p) = T_Circulation
T_DOP(p) = T_Circulation
T_DO2(p) = T_Circulation
const S₀ = buildPFD(ones(nb), DIV, Iabove)
const S′ = buildPFD(ztop, DIV, Iabove)
function T_POP(p)
    w₀, w′ = p.w₀, p.w′
    return w₀ * S₀ + w′ * S′
end
T_all = (T_DIP, T_DOP, T_POP, T_DO2)

(Main.##398.T_DIP, Main.##398.T_DOP, Main.##398.T_POP, Main.##398.T_DO2)

Because AIBECS will solve for the steady state solution directly without time-stepping the goverining equations to equilibrium, we don't have any opportunity to specify any intial conditions.
Initial conditions are how the total amount of conserved elements get specified in most global biogeochemical modelels.
Thus to specify the total inventory of P in AIBECS we add a very weak resporing term to the DIP equation.
The time-scale for this restoring term is chosen to be very long compared to the timescale with which the ocean circulation homogenizes a tracer.
Because of this long timescale we call it the geological restoring term, but geochemists who work on geological processes don't like that name!
In any event the long timescale allows us to prescribe the total inventory of P without having any appreciable impact on the 3d distribution of P.

### Sources minus sinks

##### Geological Restoring

In [6]:
function geores(x, p)
    τg, xgeo = p.τg, p.xgeo
    return (xgeo .- x) / τg
end

geores (generic function with 1 method)

##### Uptake of phosphate (DIP)

In [7]:
relu(x) = (x .≥ 0) .* x
function uptake(DIP, p)
    τu, ku, z₀ = p.τu, p.ku, p.z₀
    DIP⁺ = relu(DIP)
    return 1/τu * DIP⁺.^2 ./ (DIP⁺ .+ ku) .* (z .≤ z₀)
end

uptake (generic function with 1 method)

##### Remineralization DOP into DIP

In [8]:
function remineralization(DOP, p)
    κDOP = p.κDOP
    return κDOP * DOP
end

remineralization (generic function with 1 method)

##### Dissolution of POP into DOP

In [9]:
function dissolution(POP, p)
    κPOP = p.κPOP
    return κPOP * POP
end

dissolution (generic function with 1 method)

##### Air-sea gas exchange

In [10]:
dz1 = grd["dzt"][1]               # thickness of the top layer
z = vec(grd["ZT3d"])[iwet]        # depth of the gridbox centers
using WorldOceanAtlasTools
WOA = WorldOceanAtlasTools
μDO2 , σ²DO2 = WOA.fit_to_grid(grd,2018,"O2sat","annual","1°","an")

Expr(:const, Symbol("##bin_index_memoized_cache") = Expr(:call, :IdDict))
  ** incremental compilation may be fatally broken for this module **

Trying OPeNDAP first
  Reading NetCDF file
  Rearranging data
  Filtering data
  Averaging data over each grid box
  Setting μ = 0 and σ² = ∞ where no obs
  Setting a realistic minimum for σ²


([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.9986298060417176 0.9981694602966309 … 1.000671501159668 0.9994396591186524; 1.0175252294540404 1.0174463748931886 … 1.0179427027702332 1.0176871752738952]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.9550732585362026 0.9562181799752372 … 0.9535434395926339 0.9541520363943917; 0.9528527014596122 0.9531717082432338 … 0.9524201202392578 0.9525935690743583]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.8482290013631184 0.8513909212748211 … 0.8417107359568279 0.8448687076568604; 0.836611909866333 0.8373468748728435 … 0.8351414712270101 0.8358594926198323]

...

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.8468556912740072 0.8483416112263997 … 0.8462580108642578 0.8461976095346304; 0.8393070729573567 0.8395611953735351 … 0.8417618124825614 0.8413471494402204]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]

[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [Inf Inf … Inf Inf; Inf 

This below was commented out?

In [11]:
function airsea(DO2, p)
    κDO2 = p.κDO2
    return κDO2 * (z .< 20) .* (1.0 .- DO2) / dz1
end

airsea (generic function with 1 method)

Add them up into sms functions (Sources Minus Sinks)

In [12]:
function sms_DIP(DIP, DOP, POP, p)
    return -uptake(DIP, p) + remineralization(DOP, p) + geores(DIP, p)
end
function sms_DOP(DIP, DOP, POP, p)
    σ = p.σ
    return σ * uptake(DIP, p) - remineralization(DOP, p) + dissolution(POP, p)
end
function sms_POP(DIP, DOP, POP, p)
    σ = p.σ
    return (1 - σ) * uptake(DIP, p) - dissolution(POP, p)
end
sms_all = (sms_DIP, sms_DOP, sms_POP) # bundles all the source-sink functions in a tuple

(Main.##398.sms_DIP, Main.##398.sms_DOP, Main.##398.sms_POP)

### Parameters

Build the parameters type and p₀

In [13]:
t = empty_parameter_table()    # initialize table of parameters
add_parameter!(t, :xgeo, 2.17u"mmol/m^3",
    variance_obs = ustrip(upreferred(0.1 * 2.17u"mmol/m^3"))^2,
    description = "Geological mean P concentration",
    LaTeX = "\\state^\\mathrm{geo}")
add_parameter!(t, :τg, 1.0u"Myr",
    description = "Geological restoring timescale",
    LaTeX = "\\tau_\\mathrm{geo}")
add_parameter!(t, :ku, 10.0u"μmol/m^3",
    optimizable = true,
    description = "Half-saturation constant (Michaelis-Menten)",
    LaTeX = "k_\\vec{u}")
add_parameter!(t, :z₀, 80.0u"m",
    description = "Depth of the euphotic layer base",
    LaTeX = "z_0")
add_parameter!(t, :w₀, 1.0u"m/d",
    optimizable = true,
    description = "Sinking velocity at surface",
    LaTeX = "w_0")
add_parameter!(t, :w′, 1/4.4625u"d",
    optimizable = true,
    description = "Vertical gradient of sinking velocity",
    LaTeX = "w'")
add_parameter!(t, :κDOP, 1/0.25u"yr",
    optimizable = true,
    description = "Remineralization rate constant (DOP to DIP)",
    LaTeX = "\\kappa")
add_parameter!(t, :κPOP, 1/5.25u"d",
    optimizable = true,
    description = "Dissolution rate constant (POP to DOP)",
    LaTeX = "\\kappa")
add_parameter!(t, :σ, 0.3u"1",
    description = "Fraction of quick local uptake recycling",
    LaTeX = "\\sigma")
add_parameter!(t, :τu, 30.0u"d",
    optimizable = true,
    description = "Maximum uptake rate timescale",
    LaTeX = "\\tau_\\vec{u}")
initialize_Parameters_type(t, "Pcycle_Parameters")   # Generate the parameter type

### Generate state function and Jacobian

In [14]:
nt = length(T_all)    # number of tracers
n = nt * nb           # total dimension of the state vector
p = Pcycle_Parameters() # parameters
x = p.xgeo * ones(n) # initial iterate

800640-element Array{Float64,1}:
 0.00217
 0.00217
 0.00217
 0.00217
 0.00217
 0.00217
 0.00217
 0.00217
 0.00217
 0.00217
 ⋮      
 0.00217
 0.00217
 0.00217
 0.00217
 0.00217
 0.00217
 0.00217
 0.00217
 0.00217

F, ∇ₓF = state_function_and_Jacobian(T_all, sms_all, nb)

and solve

prob = SteadyStateProblem(F, ∇ₓF, x, p)
s = solve(prob, CTKAlg());

unpack state

function unpack_P_state(s,mask)
        DIP = NaN*mask; DOP = NaN*mask; POP = NaN*mask
        iwet = findall(x-> x==1, vec(mask))
        nwet = length(iwet)
        idip = 1:nwet;       idop = idip.+nwet;   ipop = idop.+nwet
        DIP[iwet] = s[idip]; DOP[iwet] = s[idop]; POP[iwet] = s[ipop]
    return DIP, DOP, POP
end
DIP, DOP, POP = unpack_P_state(s,wet3d);

We will plot the concentration of DIP at a given depth horizon

In [15]:
depth = vec(grd["zt"])
iz = findfirst(depth .> 200)
iz, depth[iz]

(6, 246.7350746268657)

dip = DIP[:,:,iz] * ustrip(1.0u"mol/m^3"|>u"mmol/m^3");
lat, lon = vec(grd["yt"]), vec(grd["xt"]);

and plot

ENV["MPLBACKEND"]="qt5agg"
using PyPlot, PyCall
using Conda; Conda.add("Cartopy")
clf()
ccrs = pyimport("cartopy.crs")
ax = subplot(projection=ccrs.Robinson(central_longitude=-155.0))
ax.coastlines()
# making it cyclic for Cartopy
lon_cyc = [lon; 360+lon[1]]
dip_cyc = hcat(dip, dip[:,1])
# And plot
p = contourf(lon_cyc, lat, dip_cyc, levels=0:0.2:3.6, transform=ccrs.PlateCarree(), zorder=-1)
colorbar(p, orientation="horizontal");
gcf()

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*